# Tutorial

In [1]:
# import modules: `pip install defigraph` or `python -m pip install -e .`
from defigraph.Graph import Graph
from defigraph.Edge import Edge
from defigraph.Vertex import Vertex
from defigraph.Pool import Pool

## 1.0 Get Decentralize Exchange (DEX) token data

In [2]:
# Option 1: Use data from cached results
import json

with open("data.json") as f:
    data = json.load(f)

In [19]:
# Option 2: Use data from API Call
import os
import requests
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("api_key")
base_url = "https://gateway.thegraph.com/api"
# ETH-mainnet
subgraph_id = "5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"  # Uniswap V3
subgraph_url = f"{base_url}/subgraphs/id/{subgraph_id}"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}
payload = {
    "query": "{ pools (orderBy: volumeUSD, orderDirection: desc, first: 50) { token0 { name symbol decimals id } token0Price token1Price token1 { decimals name symbol id } feeTier id } }"
}  # V3
response = requests.post(subgraph_url, headers=headers, json=payload)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()["data"]["pools"]
else:
    print("Error:", response.text)

## 1.1 Build Defi Graph

In [20]:
from web3 import Web3

edges = []

for pool in data:
    fee = pool["feeTier"]

    token0 = pool["token0"]["symbol"]
    token0_decimals = int(pool["token0"]["decimals"])
    token0_address = Web3.to_checksum_address(pool["token0"]["id"])
    token0Price = float(pool["token0Price"])

    token1 = pool["token1"]["symbol"]
    token1_decimals = int(pool["token1"]["decimals"])
    token1_address = Web3.to_checksum_address(pool["token1"]["id"])
    token1Price = float(pool["token1Price"])

    pool_address = Web3.to_checksum_address(pool["id"])

    u = Vertex(token0, token0_decimals, token0_address)
    v = Vertex(token1, token1_decimals, token1_address)

    pool = Pool(
        pool_address=pool_address,
        token0=u,
        token1=v,
        fee=fee,
        token0_price=token0Price,
        token1_price=token1Price,
    )
    pool2 = Pool(
        pool_address=pool_address,
        token0=v,
        token1=u,
        fee=fee,
        token0_price=token1Price,
        token1_price=token0Price,
    )
    edge = Edge(pool=pool)
    edge2 = Edge(pool=pool2)
    edges.append(edge)
    edges.append(edge2)

g = Graph(edges=edges)

In [21]:
print(f"No. of vertices {len(g.vertices)}")
print(f"No. of edges {len(g.get_edges())}")

No. of vertices 30
No. of edges 100


## 2. Bellman-Ford Algorithm

In [36]:
import math

all_neg_cycles = []


def bellman_ford(graph: Graph, start_vertex: Vertex):
    # 1. Initialize distances
    distances = {key: float("inf") for key in graph.vertices}
    predecessor = {key: None for key in graph.vertices}
    distances[start_vertex] = 0

    # 2. Relax edges for n - 1 steps
    for i in range(len(graph.vertices) - 1):
        for edge in g.get_edges():
            (u, v, w) = edge
            if distances[u] + w[0] < distances[v]:
                distances[v] = distances[u] + w[0]
                predecessor[v] = (u, w)

    # 3. Detect negative cycle
    for edge in g.get_edges():
        (u, v, w) = edge

        if distances[u] + w[0] < distances[v]:
            cycle = []
            # 1. add last token pool to cycle
            # 2. add 2nd last token pool to cycle
            cycle.append((w[1], w[1].address))
            cycle.append((predecessor[u][1][1], predecessor[u][1][1].address))

            # 1. use last token as start
            # 2. initialize path tracing
            # 3. initialize weight calcualtion
            # 4. start with u token predecessor
            start = v
            s = f"{v} <- {u} <- "
            c = f"{str(w[0])} + {str(predecessor[u][1][0])}"
            curr = predecessor[u][0]

            visited = set()
            visited.add(w)
            visited.add(predecessor[u][1])
            while curr != start:
                if predecessor[curr][1] in visited:
                    ###### ignoring cycles whose edge we have visted #####
                    cycle = []
                    break
                visited.add(predecessor[curr][1])
                # 1. add next path trace
                # 2. add next cycle weight
                # 3. add next token pool to path
                # 4. update next token
                s += f"{curr} <- "
                c += f" + {str(predecessor[curr][1][0])}"
                cycle.append((predecessor[curr][1][1], predecessor[curr][1][1].address))
                curr = predecessor[curr][0]

            s += f"{curr}"
            if cycle:
                print("🌟 CYCLE FOUND", w)
                print(" -> ".join(s.split(" <- ")[::-1]))
                print(f"Pools: {cycle[::-1]}")
                print(f"Profit: {math.e ** -eval(c)} = {c}\n")
                all_neg_cycles.append(cycle[::-1])
        # break
    return distances, predecessor


bellman_ford(
    g,
    Vertex(
        "WETH",
        18,
        Web3.to_checksum_address("0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"),
    ),
)
for i in all_neg_cycles:
    print(i)

🌟 CYCLE FOUND (0.005096959752283212, (USDC, UST, '100'))
UST -> USDC -> UST
Pools: [((UST, USDC, '500'), '0x92995D179a5528334356cB4Dc5c6cbb1c068696C'), ((USDC, UST, '100'), '0x18D96B617a3e5C42a2Ada4bC5d1B48e223f17D0D')]
Profit: 22.541188275053788 = 0.005096959752283212 + -3.1204411854771723

🌟 CYCLE FOUND (14.165996771154887, (WETH, HEX, '3000'))
HEX -> WETH -> HEX
Pools: [((HEX, WETH, '3000'), '0x9e0905249CeEFfFB9605E034b534544684A58BE6'), ((WETH, HEX, '3000'), '0x9e0905249CeEFfFB9605E034b534544684A58BE6')]
Profit: 1.0 = 14.165996771154887 + -14.165996771154887

🌟 CYCLE FOUND (-0.0006230051948524725, (USDT, DAI, '500'))
DAI -> USDT -> DAI
Pools: [((DAI, USDT, '100'), '0x48DA0965ab2d2cbf1C17C09cFB5Cbe67Ad5B1406'), ((USDT, DAI, '500'), '0x6f48ECa74B38d2936B02ab603FF4e36A6C0E3A77')]
Profit: 1.000140638971197 = -0.0006230051948524725 + 0.0004823761123883773

[((UST, USDC, '500'), '0x92995D179a5528334356cB4Dc5c6cbb1c068696C'), ((USDC, UST, '100'), '0x18D96B617a3e5C42a2Ada4bC5d1B48e223f17D0

In [ ]:
math.e ** -(-7.830020249256383 + 0.0009123075118552688 + 7.826887505027506)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import itertools as it

G = nx.MultiDiGraph()
print(edges)
print(g.vertices)
G.add_weighted_edges_from(edges)
plt.figure(3, figsize=(10, 10), dpi=300)

pos = nx.circular_layout(G)

nx.draw_networkx_nodes(G, pos, node_color="tab:blue", node_size=800)
connectionstyle = [f"arc3,rad={r}" for r in it.accumulate([0.04] * 4)]
nx.draw_networkx_edges(
    G,
    pos,
    edge_color="black",
    width=0.4,
    connectionstyle=connectionstyle,
    min_source_margin=5,
    min_target_margin=5,
    style="dotted",
    arrowsize=5,
    node_size=1000,
)
nx.draw_networkx_labels(
    G, pos, font_size=4, font_family="sans-serif", font_color="white"
)
nx.draw_networkx_edge_labels(
    G,
    pos,
    font_size=3,
    connectionstyle=connectionstyle,
    edge_labels={(u, v): d["weight"] for u, v, d in G.edges(data=True)},
)

plt.axis("off")
plt.show()

In [ ]:
weth = g[
    Vertex(
        "WETH",
        18,
        Web3.to_checksum_address("0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"),
    )
][0][0]
usdc = g[
    Vertex(
        "WETH",
        18,
        Web3.to_checksum_address("0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"),
    )
][0][1]